# Libraries

In [1]:
# dataframe libraries
import pandas as pd
import numpy as np

# web scraping libraries
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup
import re

# time libraries
import datetime
from dateutil.relativedelta import relativedelta
from time import sleep
import random

In [2]:
# use a new user agent
user_agent = {'User-agent': 'Mozilla/5.0'}

# Functions

In [3]:
# Classes from reddit's HTML
upvote_class = "score unvoted"
title_class = "title may-blank outbound"
url_class = "domain"
time_class = "live-timestamp"
comment_class = "first"

In [4]:
def get_post_value(soup, class_name):
    
    obj = soup.find_all(class_ = class_name)
    
    if not obj: 
        return None
    else: return obj

In [5]:
def get_upvote_list(html_list):
    upvote_list = []
    for item in html_list:
        upvotes = item.text
        if upvotes.isdigit() == True:
            upvote_list.append(int(upvotes))
        else:
            if upvotes == '•':
                upvote_list.append(0)
            else:
                upvote_list.append(int(float(upvotes[:-1])*1000))
    return upvote_list

def get_title_list(html_list):
    title_list = []
    for item in html_list:
        title_list.append(item.text)
    return title_list

def get_url_list(html_list):
    url_list = []
    for item in html_list:
        final_item = item.text.split('(')[1]
        final_item = final_item.split(')')[0]
        url_list.append(final_item)
    return url_list

def get_time_list(html_list):
    time_list = []
    for item in html_list:
        final_item = item["datetime"]
        final_item = final_item[:-6] + "Z"
        final_item = datetime.datetime.strptime(final_item, '%Y-%m-%dT%H:%M:%SZ')
        time_list.append(final_item)
    return time_list

def get_length_list(complied_times):
    length_list = []
    for item in times:
        length_list.append(datetime.datetime.utcnow()-item)
    return length_list

def get_comment_list(html_list):
    comment_list = []
    for item in html_list:
        blah = item.text.split(" ")[0]
        if blah.isdigit() == True:
            comment_list.append(int(blah))
        else:
            comment_list.append(0)
    return comment_list

# Scrape a list of urls to scrape

In [6]:
list_length = 0

# initial url
url_list = ['https://old.reddit.com/r/worldnews/new/']

# reddit only saves 31 pages
while list_length < 30:
    start_url = url_list[list_length]
    response = requests.get(start_url, headers=user_agent)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    for item in soup.find(class_ = "next-button")('a'):
        url = item['href']
    url_list.append(url)
    list_length += 1

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


# Iterate through the list of urls

In [7]:
headers2 = ['upvotes', 'title', 'site', 'time_posted', 'post_age', 'comment_no']

post_data = []

for url in url_list:
    
#     headers = {'User-agent': ua.random}
    response = requests.get(url, headers=user_agent)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    upvotes = get_post_value(soup, upvote_class)
    upvotes = get_upvote_list(upvotes)
    
    titles = get_post_value(soup, title_class)
    titles = get_title_list(titles)

    urls = get_post_value(soup, url_class)
    urls = get_url_list(urls)

    times = get_post_value(soup, time_class)
    times = get_time_list(times)

    lengths = get_length_list(times)

    comments = get_post_value(soup, comment_class)
    comments = get_comment_list(comments)
    
    for upvote, title, url, time, length, comment in zip(upvotes,titles,urls,times,lengths,comments):
        post_dict = dict(zip(headers2, [upvote,
                                title,
                                url,
                                time,
                                length,
                                comment]))
        post_data.append(post_dict)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [8]:
# Save as a dataframe
post_df = pd.DataFrame(post_data)
post_df = post_df[['upvotes', 'title', 'site', 'time_posted','post_age', 'comment_no']]

In [9]:
post_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 6 columns):
upvotes        771 non-null int64
title          771 non-null object
site           771 non-null object
time_posted    771 non-null datetime64[ns]
post_age       771 non-null timedelta64[ns]
comment_no     771 non-null int64
dtypes: datetime64[ns](1), int64(2), object(2), timedelta64[ns](1)
memory usage: 36.2+ KB


In [10]:
post_df.head()

,upvotes,title,site,time_posted,post_age,comment_no
0,0,Boycott activists fined for canceled Lorde con...,dw.com,2018-10-14 18:40:00,00:05:55.062292,1
1,0,No deal yet after flurry of Brexit diplomacy i...,reuters.com,2018-10-14 18:39:37,00:06:18.062296,1
2,0,"Saudi Arabia threatens US ""you will stab your ...",cnn.com,2018-10-14 18:38:27,00:07:28.062296,0
3,0,U.S. wants 'regime change' in Iran: Rouhani,reuters.com,2018-10-14 18:37:59,00:07:56.062297,0
4,0,"Netherlands in a ‘cyberwar’ with Russia, says ...",guardian.ng,2018-10-14 18:35:32,00:10:23.062297,0


In [11]:
# Save as a csv file
# post_df.to_csv("post_data_10_10.csv")